In [21]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy.stats import norm
from scipy.stats import ttest_ind
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression
import pickle
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [22]:
def selectkbest(indep_X,dep_Y,n):
    test= SelectKBest(f_regression, k=n)
    fit1=test.fit(indep_X,dep_Y)
    selectK_features=fit1.transform(indep_X)
    selected_col=indep_X.columns[fit1.get_support(indices=True)]
    return selectK_features, selected_col

def scalerX(indep_X):
    sc = StandardScaler()
    indep_X = sc.fit_transform(indep_X)
    return indep_X

def scalerY(dep_Y):
    scy=StandardScaler()
    dep_Y = scy.fit_transform(dep_Y)
    return dep_Y

def r2_prediction(grid,indep_X,dep_Y):
    y_pred = grid.predict(indep_X)
    r2=r2_score(dep_Y,y_pred)
    return r2

def Linear(indep_X,dep_Y):
    param_grid={'fit_intercept':[True,False],'copy_X':[True,False],'positive':[False,True]}
    grid=GridSearchCV(LinearRegression(), param_grid, refit=True, verbose=3,n_jobs=-1)
    grid.fit(indep_X,dep_Y)
    r2=r2_prediction(grid,indep_X,dep_Y)
    return  r2

def svm(indep_X,dep_Y):
    param_grid={'kernel':['linear', 'poly', 'rbf', 'sigmoid'],'C':[10,100],'gamma':['auto','scale']}
    grid=GridSearchCV(SVR(), param_grid, refit=True, verbose=3,n_jobs=-1)
    grid.fit(indep_X,dep_Y)
    r2=r2_prediction(grid,indep_X,dep_Y)
    return  r2 

def Decision(indep_X,dep_Y):
    param_grid={'criterion':['squared_error', 'friedman_mse', 'absolute_error'],'splitter':['best', 'random'],'max_features':['sqrt', 'log2']}
    grid=GridSearchCV(DecisionTreeRegressor(), param_grid, refit=True, verbose=3,n_jobs=-1)
    grid.fit(indep_X,dep_Y)
    r2=r2_prediction(grid,indep_X,dep_Y)
    return  r2 

def random(indep_X,dep_Y):
    param_grid={'n_estimators':[10,50,100],'criterion':['squared_error', 'friedman_mse', 'absolute_error'],'max_features':['sqrt', 'log2']}
    grid=GridSearchCV(RandomForestRegressor(), param_grid, refit=True, verbose=3,n_jobs=-1)
    grid.fit(indep_X,dep_Y)
    r2=r2_prediction(grid,indep_X,dep_Y)
    return  r2  

def selectk_regression(acclin,accsvm,accdes,accrf): 
    dataframe=pd.DataFrame(index=['f_regression'],columns=['Linear','SVM','Decision','Random'])
    for number,idex in enumerate(dataframe.index):        
        dataframe.loc[idex,'Linear']=acclin[number]       
        dataframe.loc[idex,'SVM']=accsvm[number]
        dataframe.loc[idex,'Decision']=accdes[number]
        dataframe.loc[idex,'Random']=accrf[number]
    return dataframe

In [23]:
dataset=pd.read_csv('sales_prep.csv',index_col=None)
df=dataset

dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

indep_X=dataset.drop('Sales_Revenue',axis=1)
dep_Y=dataset[['Sales_Revenue']]

In [24]:
kbest,selected=selectkbest(indep_X,dep_Y,5)

indep_X=scalerX(indep_X)
dep_Y=scalerY(dep_Y)

acclin=[]
accsvm=[]
accdes=[]
accrf=[]

r2_lin=Linear(indep_X,dep_Y)
acclin.append(r2_lin)
    
r2_svm=svm(indep_X,dep_Y)
accsvm.append(r2_svm)
    
r2_d=Decision(indep_X,dep_Y)
accdes.append(r2_d)
    
r2_r=random(indep_X,dep_Y)
accrf.append(r2_r)

result=selectk_regression(acclin,accsvm,accdes,accrf)

C:\Anaconda\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits


C:\Anaconda\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Anaconda\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [5]:
#2
result

,Linear,SVM,Decision,Random
f_regression,0.019711,0.990429,1.0,0.859198


In [11]:
#3
result

,Linear,SVM,Decision,Random
f_regression,0.019711,0.990429,1.0,0.84624


In [15]:
#4
result

,Linear,SVM,Decision,Random
f_regression,0.019711,0.990429,1.0,0.851863


In [26]:
#5
result

,Linear,SVM,Decision,Random
f_regression,0.019711,0.990429,1.0,0.854351


In [25]:
selected

Index(['Holiday_Season', 'Customer_Rating', 'Average_Transaction_Value',
       'Weekend_Sales_Percentage', 'Marketing_Channel_TV'],
      dtype='object')